In [1]:
import ee
import geemap
import os

## Analyzing National Land Cover Database (NLCD)

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Add NLCD data

In [3]:
dataset = ee.Image('USGS/NLCD/NLCD2016')
landcover = ee.Image(dataset.select('landcover'))
Map.addLayer(landcover, {}, 'NLCD 2016')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')

In [4]:
props = landcover.propertyNames().getInfo()
print(props)

['impervious_descriptor_class_values', 'landcover_class_names', 'system:time_end', 'system:id', 'landcover_class_values', 'system:time_start', 'impervious_descriptor_class_palette', 'system:footprint', 'impervious_descriptor_class_names', 'system:version', 'system:asset_size', 'landcover_class_palette', 'system:index', 'system:bands', 'system:band_names']


In [5]:
class_names = landcover.get('landcover_class_names').getInfo()
class_values = landcover.get('landcover_class_values').getInfo()

for index, class_name in enumerate(class_names):
    print("{}: {}\n".format(class_values[index], class_name))

11: Open Water - All areas of open water, generally with less than 25% cover or vegetation or soil.

12: Perennial Ice/Snow - All areas characterized by a perennial cover of ice and/or snow, generally greater than 25% of total cover.

21: Developed, Open Space - Includes areas with a mixture of some constructed materials, but mostly vegetation in the form of lawn grasses.  Impervious surfaces account for less than 20 percent of total cover. These areas most commonly include large-lot single-family housing units, parks, golf courses, and vegetation planted in developed settings for recreation, erosion control, or aesthetic purposes.

22: Developed, Low Intensity - Includes areas with a mixture of constructed materials and vegetation. Impervious surfaces account for 20-49 percent of total cover.  These areas most commonly include single-family housing units.

23: Developed, Medium Intensity - Includes areas with a mixture of constructed materials and vegetation. Impervious surfaces accou

### Calculate land cover compostion of each US state

In [6]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
nlcd_stats = os.path.join(out_dir, 'nlcd_stats.csv')  

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(landcover, states, nlcd_stats, statistics_type='SUM', denominator=1000000, decimal_places=2)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /home/qiusheng/Downloads/nlcd_stats.csv


## Analyzing Global Land Cover

In [7]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Add MODIS global land cover data

MODIS MCD12Q1.051 Land Cover Type Yearly Global 500m 

https://developers.google.com/earth-engine/datasets/catalog/MODIS_051_MCD12Q1

In [8]:
landcover = ee.Image('MODIS/051/MCD12Q1/2013_01_01') \
    .select('Land_Cover_Type_1')

Map.setCenter(6.746, 46.529, 2)
Map.addLayer(landcover, {}, 'MODIS Land Cover')

In [9]:
props = landcover.propertyNames().getInfo()
print(props)

['Land_Cover_Type_4_class_names', 'Land_Cover_Type_5_class_names', 'Land_Cover_Type_1_class_palette', 'Land_Cover_Type_2_class_palette', 'Land_Cover_Type_2_class_values', 'Land_Cover_Type_3_class_names', 'system:time_end', 'Land_Cover_Type_4_class_values', 'Land_Cover_Type_1_class_names', 'Land_Cover_Type_3_class_values', 'system:id', 'Land_Cover_Type_3_class_palette', 'Land_Cover_Type_2_class_names', 'Land_Cover_Type_5_class_palette', 'Land_Cover_Type_5_class_values', 'system:time_start', 'Land_Cover_Type_4_class_palette', 'system:footprint', 'system:version', 'Land_Cover_Type_1_class_values', 'system:asset_size', 'system:index', 'system:bands', 'system:band_names']


In [10]:
class_names = landcover.get('Land_Cover_Type_1_class_names').getInfo()
class_values = landcover.get('Land_Cover_Type_1_class_values').getInfo()

for index, class_name in enumerate(class_names):
    print("{}: {}".format(class_values[index], class_name))

0: Water
1: Evergreen Needleleaf forest
2: Evergreen Broadleaf forest
3: Deciduous Needleleaf forest
4: Deciduous Broadleaf forest
5: Mixed forest
6: Closed shrublands
7: Open shrublands
8: Woody savannas
9: Savannas
10: Grasslands
11: Permanent wetlands
12: Croplands
13: Urban and built-up
14: Cropland/Natural vegetation mosaic
15: Snow and ice
16: Barren or sparsely vegetated
254: Unclassified


In [11]:
countries_shp = '../data/countries.shp'
countries = geemap.shp_to_ee(countries_shp)
Map.addLayer(countries, {}, 'Countries')

In [12]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
global_stats = os.path.join(out_dir, 'global_stats.csv')  

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(landcover, countries, global_stats, statistics_type='PERCENTAGE', denominator=1000000, decimal_places=2)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /home/qiusheng/Downloads/global_stats.csv
